# Creates kerchunks from specified pattern or files on Azure Blobs

In [ ]:
#!pip install autopep8

import fsspec
import ujson
import xarray as xr
from kerchunk.combine import MultiZarrToZarr
from kerchunk.hdf import SingleHdf5ToZarr
from tqdm import tqdm

In [ ]:
import adlfs

print(adlfs.__version__)

In [ ]:
import os
#os.environ['AZURE_STORAGE_ACCOUNT_KEY']

In [ ]:
import kerchunk_dask

In [ ]:
from kerchunk_dask import retry_on_failure

SingleHdf5ToZarr.translate = retry_on_failure()(SingleHdf5ToZarr.translate)

MultiZarrToZarr.translate = retry_on_failure()(MultiZarrToZarr.translate)

from kerchunk import combine

combine.MultiZarrToZarr.translate = retry_on_failure()(combine.MultiZarrToZarr.translate)

## Start dask cluster

In [ ]:
import dask

import logging

from distributed import Client

client = Client(n_workers=24, silence_logs=logging.ERROR)
client

## Needs to be improved... i.e. obtained from other credentials

In [ ]:
# needs to be improved
account_dict = dict(account_name = "<get this from the azure portal: storage account name>",
               account_key="<get this from the azure portal for this account name: key>")

In [ ]:
#urls = kerchunk_dask.glob_urls('abfs://bay-delta-schism2-v58/eli/simulations/hindcast_clinic2/outputs/schout_0001_*.nc',
#                               account_dict)
urls = [f'abfs://bay-delta-schism2-v58/eli/simulations/hindcast_clinic2/outputs/schout_0005_{i}.nc' for i in range(1,3948)]

In [ ]:
urls[0:3],len(urls)

In [ ]:
#import fsspec
#import xarray as xr
#fs_read = fsspec.filesystem('abfs',**account_dict)
#ds=xr.open_dataset(fs_read.open(urls[0]))
#ds=ds.assign_coords({'lon':ds['lon'],'lat':ds['lat']})
#ds

In [ ]:
mzz_json_filename = '_'.join(urls[0].split('/')[-1].split('_')[0:-1])+'.json'

In [ ]:
from kerchunk.combine import auto_dask

auto_dask(urls, single_driver=SingleHdf5ToZarr, 
          single_kwargs = {'storage_options': account_dict }, 
          mzz_kwargs = { #'coo_map':{'time':'cf:time'},
                        'concat_dims':['time'] ,
                        'identical_dims':['nSCHISM_hgrid_node', 'nSCHISM_vgrid_layers','two','nSCHISM_hgrid_face']}, 
          n_batches=24, remote_protocol='abfs', remote_options=account_dict,
          filename = mzz_json_filename)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import hvplot.xarray

In [ ]:
backend_args = { "consolidated": False, 
                "storage_options": {"fo": mzz_json_filename,
                                    "remote_protocol": "abfs","remote_options": account_dict,
                                    "default_fill_cache": True, "default_cache_type":"mmap"}
               }

In [ ]:
ds = xr.open_dataset("reference://", backend_kwargs=backend_args, engine="zarr", chunks={})# 'auto'

In [ ]:
ds

In [ ]:
ds.elev.isel(time=slice(0,1000),nSCHISM_hgrid_node=1000).plot()

In [ ]:
client.shutdown()

## See other notebook to read using the combined files

[using combined json files to read](./azure_kerchunk_read_combined.ipynb)